# Note-Saving Tool with Pydantic and Anthropic Tool Use

In this example, we'll create a tool that saves a note with the author and metadata, and use Pydantic to validate the tool inputs. We'll define Pydantic models for validation and use the SDK's tool runner to automatically handle the agentic loop.

## Step 1: Set up the environment
First, let's install the required libraries and set up the Claude API client.

In [ ]:
%pip install anthropic pydantic 'pydantic[email]'

In [ ]:
from anthropic import Anthropic, beta_tool
from pydantic import BaseModel, EmailStr, Field

client = Anthropic()
MODEL_NAME = "claude-sonnet-4-5-20250929"

## Step 2: Define the Pydantic models

We'll define Pydantic models to represent the expected schema for the note, author, and the model's response. This will allow us to validate and type-check the model's response when saving a note.

In [18]:
class Author(BaseModel):
    name: str
    email: EmailStr


class Note(BaseModel):
    note: str
    author: Author
    tags: list[str] | None = None
    priority: int = Field(ge=1, le=5, default=3)
    is_public: bool = False


class SaveNoteResponse(BaseModel):
    success: bool
    message: str

## Step 3: Define the tool with Pydantic validation

We'll define the `save_note` tool using the `@beta_tool` decorator. The decorator extracts the tool schema from the function signature and docstring. Inside the function, we use Pydantic models to validate the inputs.

In [ ]:
@beta_tool
def save_note(
    note: str, author_name: str, author_email: str, priority: int = 3, is_public: bool = False
) -> str:
    """A tool that saves a note with the author and metadata.

    Args:
        note: The content of the note to be saved.
        author_name: The name of the author.
        author_email: The email address of the author.
        priority: The priority level of the note (1-5). Defaults to 3.
        is_public: Indicates whether the note is publicly accessible. Defaults to False.

    Returns:
        A JSON string indicating success or failure.
    """
    try:
        # Validate inputs using Pydantic models
        validated_author = Author(name=author_name, email=author_email)
        validated_note = Note(
            note=note,
            author=validated_author,
            priority=priority,
            is_public=is_public,
        )

        # In a real application, you would save the note to a database here
        print(f"Note saved successfully: {validated_note.note[:50]}...")

        response = SaveNoteResponse(success=True, message="Note saved successfully!")
        return response.model_dump_json()

    except Exception as e:
        response = SaveNoteResponse(success=False, message=f"Validation error: {str(e)}")
        return response.model_dump_json()

## Step 4: Interact with the chatbot using the Tool Runner

Now, let's create a function to interact with the chatbot using the SDK's tool runner. The tool runner automatically handles the agentic loop - executing tools when Claude requests them and continuing the conversation until Claude provides a final response.

In [ ]:
def chatbot_interaction(user_message):
    print(f"\n{'=' * 50}\nUser Message: {user_message}\n{'=' * 50}")

    # Use the tool runner to handle the agentic loop automatically
    runner = client.beta.messages.tool_runner(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=[save_note],
        messages=[{"role": "user", "content": user_message}],
    )

    # Iterate through responses - the runner executes tools automatically
    for message in runner:
        print(f"\nResponse (stop_reason: {message.stop_reason}):")
        for block in message.content:
            if hasattr(block, "text"):
                print(f"Text: {block.text}")
            elif block.type == "tool_use":
                print(f"Tool Used: {block.name}")
                print(f"Tool Input: {block.input}")

    # The final message contains Claude's response after all tool use
    final_response = next(
        (block.text for block in message.content if hasattr(block, "text")),
        None,
    )
    print(f"\nFinal Response: {final_response}")
    return final_response

## Step 5: Test the chatbot
Let's test our chatbot with a sample query to save a note.

In [26]:
chatbot_interaction("""
Can you save a private note with the following details?
Note: Remember to buy milk and eggs.
Author: John Doe (johndoe@gmail.com)
Priority: 4
""")


User Message: 
Can you save a private note with the following details?
Note: Remember to buy milk and eggs.
Author: John Doe (johndoe@gmail.com)
Priority: 4


Initial Response:
Stop Reason: tool_use
Content: [ContentBlock(text='<thinking>\nThe relevant tool to use here is save_note, as the request is to save a note with specific details.\n\nLet\'s go through the parameters one-by-one:\n\nnote: The user provided the note content: "Remember to buy milk and eggs."\nauthor: The user provided the author details: \n{\n  "name": "John Doe",\n  "email": "johndoe@gmail.com"\n}\nis_public: While the user didn\'t explicitly specify, they asked for a "private note", so we can infer is_public should be false.\npriority: The user specified a priority of 4.\n\nAll the required parameters have been provided or can be reasonably inferred from the request. We have enough information to make the save_note call.\n</thinking>', type='text'), ContentBlockToolUse(id='toolu_015iteV2eC1C7aUodbkotfiS', input={

'Your private note has been saved successfully with the following details:\n\nNote: Remember to buy milk and eggs. \nAuthor: John Doe (johndoe@gmail.com)\nPriority: 4\nVisibility: Private\n\nPlease let me know if you need anything else!'

In this example, we've created a tool that saves a note with the author and metadata using the SDK's tool runner. The tool runner automatically handles the agentic loop - executing tools when Claude requests them and continuing the conversation.

Pydantic is used inside the tool function to validate inputs. The `Author`, `Note`, and `SaveNoteResponse` models ensure that inputs conform to the expected schema. If validation fails, a friendly error message is returned.

By combining `@beta_tool` decorated functions with Pydantic validation, we get both automatic tool schema generation and robust input validation.